In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from public_tests import *

%matplotlib inline

# **Problem Statement**
Suppose you are starting a company that grows and sells wild mushrooms.

*   Since not all mushrooms are edible, you'd like to be able to tell whether a given mushroom is edible or poisonous based on it's physical attributes.

*   You have some existing data that you can use for this task.

Can you use the data to help you identify which mushrooms can be sold safely?

In [10]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])
root_indices = np.arange(len(X_train))
X_train.shape, y_train.shape

((10, 3), (10,))

- `X_train` contains three features for each example
    - Brown Color (A value of `1` indicates "Brown" cap color and `0` indicates "Red" cap color)
    - Tapering Shape (A value of `1` indicates "Tapering Stalk Shape" and `0` indicates "Enlarging" stalk shape)
    - Solitary  (A value of `1` indicates "Yes" and `0` indicates "No")

- `y_train` is whether the mushroom is edible
    - `y = 1` indicates edible
    - `y = 0` indicates poisonous

<a name="ex01"></a>
### Exercise 1

Please complete the `compute_entropy()` function using the previous instructions.

In [3]:
# The log is calculated with base  2
# For implementation purposes,  0log2(0)=0 . That is, if p_1 = 0 or p_1 = 1, set the entropy to 0
# Make sure to check that the data at a node is not empty (i.e. len(y) != 0). Return 0 if it is

def compute_entropy(y):
  if len(y) ==0:
    return 0
  else:
    num_edible = np.count_nonzero(y == 1)
    p = num_edible / len(y)
    if p == 0 or p ==1:
      return 0
    else:
      return -p * np.log2(p) - (1 - p) * np.log2(1 - p)

<a name="ex02"></a>
### Exercise 2

Please complete the `split_dataset()` function shown below

- For each index in `node_indices`
    - If the value of `X` at that index for that feature is `1`, add the index to `left_indices`
    - If the value of `X` at that index for that feature is `0`, add the index to `right_indices`

In [6]:
def split_dataset(X, node_indices, feature):
  left_indices = []
  right_indices = []
  for i in node_indices:
    if X[i,feature] == 1:
      left_indices.append(i)
    else:
      right_indices.append(i)
  return left_indices, right_indices

<a name="ex03"></a>
### Exercise 3

Please complete the `compute_information_gain()` function shown below to compute

$$\text{Information Gain} = H(p_1^\text{node})- (w^{\text{left}}H(p_1^\text{left}) + w^{\text{right}}H(p_1^\text{right}))$$

where
- $H(p_1^\text{node})$ is entropy at the node
- $H(p_1^\text{left})$ and $H(p_1^\text{right})$ are the entropies at the left and the right branches resulting from the split
- $w^{\text{left}}$ and $w^{\text{right}}$ are the proportion of examples at the left and right branch respectively

Note:
- You can use the `compute_entropy()` function that you implemented above to calculate the entropy
- We've provided some starter code that uses the `split_dataset()` function you implemented above to split the dataset

In [8]:
def compute_information_gain(X,y,node_indices, feature):
  left_indices, right_indices = split_dataset(X, node_indices, feature)

  w_left = len(left_indices) / len(node_indices)
  w_right = len(right_indices) / len(node_indices)

  H_left = compute_entropy(y[left_indices])
  H_right = compute_entropy(y[right_indices])
  H_node = compute_entropy(y[node_indices])

  information_gian = H_node - (w_left * H_left + w_right * H_right)
  return information_gian

In [11]:
# feature 0: Brown Color (A value of 1 indicates "Brown" cap color and 0 indicates "Red" cap color)
Browncolor_information_gain = compute_information_gain(X_train, y_train, root_indices,0)
print(f"Information Gain for Brown Color: {Browncolor_information_gain}")

# feature 1: Tapering Shape (A value of 1 indicates "Tapering Stalk Shape" and 0 indicates "Enlarging" stalk shape)
Taperingshape_information_gain = compute_information_gain(X_train, y_train,root_indices, 1)
print(f"Information Gain for Tapering Shape: {Taperingshape_information_gain}")

# feature 2: Solitary (A value of 1 indicates "Yes" and 0 indicates "No") -- maximum information gain
Solitary_information_gain = compute_information_gain(X_train, y_train,root_indices, 2)
print(f"Information Gain for Solitary: {Solitary_information_gain}")

Information Gain for Brown Color: 0.034851554559677034
Information Gain for Tapering Shape: 0.12451124978365313
Information Gain for Solitary: 0.2780719051126377


<a name="ex04"></a>
### Exercise 4
Please complete the `get_best_split()` function shown below.
- The function takes in the training data, along with the indices of datapoint at that node
- The output of the function the feature that gives the maximum information gain
    - You can use the `compute_information_gain()` function to iterate through the features and calculate the information for each feature

In [15]:
def get_best_split(X, y, node_indices):
  best_features = -1
  max_information_gain = 0
  feature = X.shape[1]

  for i in range(feature):
    information_gain = compute_information_gain(X,y,node_indices, i)
    if max_information_gain < information_gain:
      best_features = i
      max_information_gain = information_gain
    else:
      continue

  return best_features

In [16]:
best_feature = get_best_split(X_train, y_train, root_indices)
print(f"Best Feature: {best_feature}")

Best Feature: 2


<a name="5"></a>
## Building the tree

In this section, we use the functions you implemented above to generate a decision tree by successively picking the best feature to split on until we reach the stopping criteria (maximum depth is 2).

In [19]:
# Not graded
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.

    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree.
        current_depth (int):    Current depth. Parameter used during recursive call.

    """
    if current_depth == max_depth:
      formatting = ''* current_depth + '-'* current_depth
      print(formatting, "%s leaf node with indices" % branch_name, node_indices)
      return

    best_feature = get_best_split(X,y,node_indices)
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((current_depth, branch_name, best_feature, node_indices))
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))

    build_tree_recursive(X, y, left_indices, 'Left', max_depth, current_depth + 1)
    build_tree_recursive(X, y, right_indices, 'Right', max_depth, current_depth + 1)


In [21]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
-- Left leaf node with indices [0, 1, 4, 7]
-- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
-- Left leaf node with indices [8]
-- Right leaf node with indices [2, 3, 6, 9]
